# Winograd algorithm for 3D convolution
## Background

The Winograd family of fast convolution algorithms attempts to minimize the number of operations needed for fixed-size small convolutions. Around 1980, Winograd proved that a convolution of the input of length $n$ with a kernel of length $n_{h}$ can be computed using a theoretical minimum of just $n+n_{h}-1$ general multiplications.

However, most studies actually used Toom-Cook algorithm and described their approach as "Winograd convolution" and within the DNN research literature that term has come to include both Toom-Cook and Winograd methods. Toom-Cook convolution algorithm based on the Chinese Reminder Theorem for polynomials and the Matrix Exchange Theorem.

**Toom-Cook algorithm.** The one-dimensional discrete convolution of two vectors $h[h_{1}..h_{n_{h}}]$ and $x[x_{1}..x_{n}]$ is the vector $s = h*x$ where $s_{i}= \sum \limits_{j=1}^{i} h_{j}*x_{i-j}$ . Using Toom-Cook algorithm we can present it as $(h*x)=V^{-1}(V_{x}x \odot V_{h}h)$ where $V_{x}$ and $V_{h}$ are *Vandermonde* matrices to transform values into the *modulo polynomial domain* and evaluate polynomials in different $p_{i}$ points, and $V^{-1}$ is the *inverse Vandermonde* matrix for the transformation back from the *modulo polynomial domain*.

$$ (h*x) = V^{-1}(V_{x}x \odot V_{h}h) = V_{x}^{T}(V_{h}h \odot V^{-T}x) $$

Let $A=V_{x}, G=V_{h}, B=V^{-1}$

$$ (h*x)_{1D} = A^{T}(Gh \odot B^{T}x) $$

In similar way for 2D-convolution:
$$ (h*x)_{2D} = A^{T}(GhG^{T} \odot B^{T}xB)A $$

**N-mode Tensor-Matrix product.** *The n-mode Tensor-Matrix product* of a tensor $X \in \R^{I_{1} \times I_{2} \times ... \times I_{N}}$ with a matrix $U \in \R^{J \times I_{n}}$ is denoted by $M = X  \times_{n} U$ and $M \in \R^{I_{1} \times ...J \times ... \times I_{N}}$.

$$(X  \times_{n} U)_{i_{1}...j...i_{N}} = \sum \limits_{i_{n}=1}^{I_{n}} X_{i_{1}...i_{n}...i_{N}} \times U_{ji_{n}}$$

For matrices *X* and *U* n-mode Tensor-Matrix product is equivalent to:
$$ X  \times_{1} U = UX$$
$$ X  \times_{2} U = XU^{T}$$
$$ (X  \times_{1} U) \times_{2} = X  \times_{1} U \times_{2} = UXU^{T}$$

In [1]:
import numpy as np

def nmod_tensor_product(A, U, n):
    """
    n-Mode Tensor - Matrix Product
    ref: https://arxiv.org/pdf/1611.06565.pdf

    Input:
    A: tensor of R^( I_1 × I_2 × .. I_n × .. I_N )
    U: matrix of R^( J × I_n)
    n: scalar within [1:N], specifying the mode

    Output:
    B: output tensor of R^(I_1 × I_2 × .. J × .. I_N)

    Calculation:
    M[i1,i2,..,j,..,iN] = sum(A[i1,i2,..,in,..,iN] * U[j,in]) by axis 'in'
    """
    assert n > 0 and n <= len(A.shape)
    assert A.shape[n-1] == U.shape[1]

    i_n = A.shape[n-1]
    j = U.shape[0]

    M_shape = np.asarray(A.shape)
    M_shape[n-1] = j
    M = np.zeros(M_shape)

    for idx, _ in np.ndenumerate(M):
        j_idx = idx[n-1]

        acc = 0.0
        for i in range(i_n):
            A_idx = np.asarray(idx)
            A_idx[n-1] = i
            acc += A[tuple(A_idx)] * U[j_idx, i]

        M[idx] = acc

    return M

## N-dimensional Winograd algorithm

Using *n-mode Tensor-Matrix product* we can extend Toom-Cook algorithm for *N-dimensional* tensor:
$$ (h*x)_{ND} = ((h \times_{1} G ... \times_{N} G) \odot (x \times_{1} B^{T} ... \times_{N} B^{T})) \times_{1} A^{T} ... \times_{N} A^{T} $$

In [16]:
def conv_wino3d(data, filter, AT, G, BT):
    d_transform = nmod_tensor_product(
        nmod_tensor_product(
            nmod_tensor_product(d, BT, 1),
            BT, 2),
        BT, 3)

    f_transform = nmod_tensor_product(
        nmod_tensor_product(
            nmod_tensor_product(f, G, 1),
            G, 2),
        G, 3)

    accum = np.multiply(d_transform, f_transform)
    x_back_transform = nmod_tensor_product(accum, AT, 1)
    y_back_transform = nmod_tensor_product(x_back_transform, AT, 2)
    z_back_transform = nmod_tensor_product(y_back_transform, AT, 3)
    return z_back_transform

def conv_winoNd(data, filter, AT, G, BT, N):
    assert N > 0

    dim_range = range(2, N+1)

    d_transform = nmod_tensor_product(d, BT, 1)
    for n in dim_range:
        d_transform = nmod_tensor_product(d_transform, BT, n)

    f_transform = nmod_tensor_product(f, G, 1)
    for n in dim_range:
        f_transform = nmod_tensor_product(f_transform, G, n)

    accum = np.multiply(d_transform, f_transform)
    
    back_transform = nmod_tensor_product(accum, AT, 1)
    for n in dim_range:
        back_transform = nmod_tensor_product(back_transform, AT, n)

    return back_transform

## References

1. Blahut R.E. Fast algorithms for signal processing
2. Fast Algorithms for Convolutional Neural Networks, [URL](https://arxiv.org/pdf/1509.09308.pdf)
3. Error Analysis and Improving the Accuracy of Winograd Convolution for Deep Neural Networks, [URL](https://arxiv.org/pdf/1803.10986.pdf)
4. Tensor Decompositions and Applications, [URL](https://epubs.siam.org/doi/10.1137/07070111X)
5. Deep Tensor Convolution on Multicores, [URL](https://arxiv.org/pdf/1611.06565.pdf)
6. High Performance Implementation of 3D Convolutional Neural Networks on a GPU, [URL](https://www.hindawi.com/journals/cin/2017/8348671/)
7. Learning Spatiotemporal Features with 3D Convolutional Networks (2015), [URL](https://arxiv.org/pdf/1412.0767.pdf)
8. Quo Vadis, Action Recognition? A New Model and the Kinetics Dataset (2018), [URL](https://arxiv.org/pdf/1705.07750.pdf)
9. A Closer Look at Spatiotemporal Convolutions for Action Recognition (2018), [URL](https://openaccess.thecvf.com/content_cvpr_2018/papers/Tran_A_Closer_Look_CVPR_2018_paper.pdf)
10. Video Classification with Channel-Separated Convolutional Networks (2019), [URL](https://arxiv.org/pdf/1904.02811.pdf)